In [11]:
df = spark.sql("SELECT * FROM Bing_LH.tbl_latest_news")
display(df)

StatementMeta(, dd873f50-171b-4cc0-b3ff-927395e26e1d, 13, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 5b218d3e-3d0e-45e2-8b0e-7cce460ed596)

In [12]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

StatementMeta(, dd873f50-171b-4cc0-b3ff-927395e26e1d, 14, Finished, Available, Finished)

In [13]:
#Import the model and configure the input and output columns
model = (AnalyzeText()
        .setTextCol("description")
        .setKind("SentimentAnalysis")
        .setOutputCol("response")
        .setErrorCol("error"))

StatementMeta(, dd873f50-171b-4cc0-b3ff-927395e26e1d, 15, Finished, Available, Finished)

In [14]:
#Apply the model to our dataframe
result = model.transform(df)

StatementMeta(, dd873f50-171b-4cc0-b3ff-927395e26e1d, 16, Finished, Available, Finished)

In [15]:
display(result)

StatementMeta(, dd873f50-171b-4cc0-b3ff-927395e26e1d, 17, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b802be87-aed6-45df-a895-4aa9e0a01dec)

In [16]:
#Create Sentiment Column
from pyspark.sql.functions import col

sentiment_df = result.withColumn("sentiment", col("response.documents.sentiment"))

StatementMeta(, dd873f50-171b-4cc0-b3ff-927395e26e1d, 18, Finished, Available, Finished)

In [17]:
display(sentiment_df)

StatementMeta(, dd873f50-171b-4cc0-b3ff-927395e26e1d, 19, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d85e261a-3060-409e-9d2e-36b435bb5f39)

In [18]:
sentiment_df_final = sentiment_df.drop("error","response")
display(sentiment_df_final)

StatementMeta(, dd873f50-171b-4cc0-b3ff-927395e26e1d, 20, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 9cb99266-ef40-463d-9267-9940d1b2a855)

In [19]:
from pyspark.sql.functions import col, to_date

sentiment_df_final = sentiment_df_final.withColumn("datePublished", to_date(col("datePublished"), "dd-MMM-yyyy"))

StatementMeta(, dd873f50-171b-4cc0-b3ff-927395e26e1d, 21, Finished, Available, Finished)

In [20]:
sentiment_df_final.printSchema()

StatementMeta(, dd873f50-171b-4cc0-b3ff-927395e26e1d, 22, Finished, Available, Finished)

root
 |-- name: string (nullable = true)
 |-- description: string (nullable = true)
 |-- category: string (nullable = true)
 |-- url: string (nullable = true)
 |-- image: string (nullable = true)
 |-- provider: string (nullable = true)
 |-- datePublished: date (nullable = true)
 |-- sentiment: string (nullable = true)



In [21]:
from pyspark.sql.utils import AnalysisException

try:

    table_name = 'Bing_LH.sentiment_analysis_table'

    sentiment_df_final.write.format("delta").saveAsTable(table_name)

except AnalysisException:

    print("Table Already Exists")

    sentiment_df_final.createOrReplaceTempView("vw_sentiment_df_final")

    spark.sql(f"""  MERGE INTO {table_name} target_table
                    USING vw_sentiment_df_final source_view

                    ON source_view.url = target_table.url
                    
                    WHEN MATCHED AND 
                    source_view.name<> target_table.name OR
                    source_view.description <> target_table.description OR
                    source_view.category <> target_table.category OR
                    source_view.image <> target_table.image OR
                    source_view.provider <> target_table.provider OR
                    source_view.datePublished <> target_table.datePublished   

                    THEN UPDATE SET *

                    WHEN NOT MATCHED THEN INSERT *

                """)

StatementMeta(, dd873f50-171b-4cc0-b3ff-927395e26e1d, 23, Finished, Available, Finished)